In [1]:
import re
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Lambda, Input
from keras.optimizers import SGD
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.datasets import load_boston
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import numpy as np
import pandas as pd

Using TensorFlow backend.


In [3]:
sdb = pd.read_csv("Shakespeare-plays/Shakespeare_data.csv")

In [5]:
class_names = ['comedy','tragedy','other']

In [9]:
def all_the_plays():
    new_play = "Henry IV"
    play_array = []
    for i in sdb['Play']:
        if i != new_play:
            play_array.append(new_play)
            new_play = i
    return play_array

In [10]:
all_plays = all_the_plays()

In [12]:
def get_text(play_name):
    play = sdb[sdb['Play']==play_name]
    w = play['PlayerLine']
    text = ""
    for i in w:
        text += i
        text += " "
    return text

In [94]:
comedy_training = []
tragedy_training = []
history_training = []

In [95]:
def get_training_text():
    comedy_training.append(get_text("Alls well that ends well"))
    comedy_training.append(get_text("As you like it"))
    comedy_training.append(get_text("Loves Labours Lost"))
    comedy_training.append(get_text("Measure for measure"))
    comedy_training.append(get_text("Merry Wives of Windsor"))
    comedy_training.append(get_text("A Midsummer nights dream"))
    comedy_training.append(get_text("Taming of the Shrew"))
    comedy_training.append(get_text("The Tempest"))
    comedy_training.append(get_text("Two Gentlemen of Verona"))
    comedy_training.append(get_text("A Winters Tale"))
    tragedy_training.append(get_text("Antony and Cleopatra"))
    tragedy_training.append(get_text("Coriolanus"))
    tragedy_training.append(get_text("Hamlet"))
    tragedy_training.append(get_text("Julius Caesar"))
    tragedy_training.append(get_text("macbeth"))
    tragedy_training.append(get_text("Othello"))
    tragedy_training.append(get_text("Timon of Athens"))
    tragedy_training.append(get_text("Titus Andronicus"))
    history_training.append(get_text("Henry IV"))
    history_training.append(get_text("Henry VI Part 1"))
    history_training.append(get_text("Henry VI Part 3"))
    history_training.append(get_text("Henry VIII"))
    history_training.append(get_text("King John"))
    history_training.append(get_text("Richard II"))

In [96]:
comedy_testing = []
tragedy_testing = []
history_testing = []

In [97]:
def get_testing_text():
    comedy_testing.append(get_text("A Comedy of Errors"))
    comedy_testing.append(get_text("Merchant of Venice"))
    comedy_testing.append(get_text("Much Ado about nothing"))
    comedy_testing.append(get_text("Twelfth Night"))
    tragedy_testing.append(get_text("Cymbeline"))
    tragedy_testing.append(get_text("King Lear"))
    tragedy_testing.append(get_text("Romeo and Juliet"))
    tragedy_testing.append(get_text("Troilus and Cressida"))
    history_testing.append(get_text("Henry V"))
    history_testing.append(get_text("Henry VI Part 3"))
    history_testing.append(get_text("Pericles"))
    history_testing.append(get_text("Richard III"))

In [98]:
get_training_text()
get_testing_text()

In [102]:
def get_shakes_text_array(text):
    w = re.split("\W*[^\'\w+\']", text)
    text_array = []
    for i in w:
        text_array.append([i])
    return text_array